In [8]:
from IPython.display import display, Markdown

planner_markdown = """# Step-by-Step Plan

## Plan Steps

| Agent | Task | Stopping Criteria |
|-------|------|-------------------|
{plan_str}
"""
plan=[("Agent1", "Task1", "Criteria1"), ("Agent2", "Task2", "Criteria2")]
plan_str = '\n'.join([f'|{agent_name}|{task}|{stopping_criteria}|' for agent_name, task, stopping_criteria in plan])
test = planner_markdown.format(plan_str=plan_str)
display(Markdown(test))

# Step-by-Step Plan

## Plan Steps

| Agent | Task | Stopping Criteria |
|-------|------|-------------------|
|Agent1|Task1|Criteria1|
|Agent2|Task2|Criteria2|


In [13]:


response = {'router': {
    "next_agent": "Agent1",
    "next_task": "Task1",
    "stopping_criteria": "Criteria1"
    }}

router_markdown = f"""
    # 📋 Task Assignment

    ---

    ### 👤 Next Agent
    **{response['router']['next_agent']}**

    ---

    ### 🎯 Next Task
    **{response['router']['next_task']}**

    ---

    ### ⏹️ Stopping Criteria
    **{response['router']['stopping_criteria']}**
    """
display(Markdown(router_markdown))


    # 📋 Task Assignment

    ---

    ### 👤 Next Agent
    **Agent1**

    ---

    ### 🎯 Next Task
    **Task1**

    ---

    ### ⏹️ Stopping Criteria
    **Criteria1**
    

In [4]:
plan=[("Agent1", "Task1", "Criteria1"), ("Agent2", "Task2", "Criteria2")]
print(''.join([f'{agent_name}/{task}/{stopping_criteria}/\n' for agent_name, task, stopping_criteria in plan]))

Agent1/Task1/Criteria1/
Agent2/Task2/Criteria2/



In [14]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver



class State(TypedDict):
    input: str
    user_feedback: str


def step_1(state):
    print("---Step 1---")
    pass


def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}


def step_3(state):
    print("---Step 3---")
    pass


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

# Input
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

---Step 1---
{'step_1': None}


---human_feedback---
{'__interrupt__': (Interrupt(value='Please provide feedback:', resumable=True, ns=['human_feedback:1842b1fa-954e-31b7-b1de-52448fd7278d'], when='during'),)}




In [15]:
# Continue the graph execution
for event in graph.stream(
    Command(resume="go to step 3!"), thread, stream_mode="updates"
):
    print(event)
    print("\n")

---human_feedback---
{'human_feedback': {'user_feedback': 'go to step 3!'}}


---Step 3---
{'step_3': None}




In [15]:
from langchain_core.messages import SystemMessage
SystemMessage(content="Testing").content

'Testing'